In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import os
FOLDER_NAME = 'dataset_extracted'

CLASSES = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [ ]:
# Create label map
label_map = {}
for (root, folders, files) in os.walk(FOLDER_NAME):
    for foldername in folders:
        if foldername in CLASSES:
            label_map[foldername] = CLASSES.index(foldername)

print(label_map)

In [ ]:
sequence, label = [], []
target_length = 48
for (root, folders, files) in os.walk(FOLDER_NAME):
    for filename in files:
        file_path = os.path.join(os.path.relpath(
            root, FOLDER_NAME), filename)
        if(os.path.split(file_path)[0] in CLASSES):
            res = np.load(f'{FOLDER_NAME}/{file_path}')
            for _ in range(target_length-res.shape[0]):
                res = np.vstack((res, res[-1, :]))
            sequence.append(np.array(res))
            label.append(label_map[os.path.basename(root)])

print(np.array(sequence).shape)
print(np.array(label).shape)


In [ ]:
# FOR DATA DUPLICATION
# sequence = np.concatenate([sequence, sequence, sequence, ], axis=0)
# label = np.concatenate([label, label, label], axis=0)

# print(np.array(sequence).shape)
# print(np.array(label).shape)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(sequence), tf.keras.utils.to_categorical(
    np.array(label).astype(int), num_classes=np.array(CLASSES).shape[0], dtype='float32'), test_size=0.05)

print(X_train.shape, X_test.shape)


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard


In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)


In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True,
          activation='tanh', input_shape=(48, 1662)))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(LSTM(64, return_sequences=False, activation='tanh'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(np.array(CLASSES).shape[0], activation='softmax'))


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])


In [ ]:
tf.config.list_physical_devices('GPU')


In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])


In [ ]:
model.summary()


In [ ]:
res = model.predict(X_test)